In [1]:
import sys
import os
sys.path.append(os.path.abspath("../../../../"))

import pandas as pd
import numpy as np
import utils
# Classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# Evaluation
from sklearn.metrics import classification_report
import plot_automations as plotter

In [2]:
df = pd.read_csv("../../../../01_color_clustering/weighted/weighted_omniart-post-color-clustering.csv")

In [3]:
# Add decade to creation intervals
df["decade"] = (df["creation_year"] // 10 * 10).astype(int)

In [4]:
color_clusters = pd.read_csv("../../../../01_color_clustering/weighted/weighted_color_centroids.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../../../01_color_clustering/weighted/weighted_color_centroids.csv'

**1. Features**

In [ ]:
color_names = color_clusters["HEX"].tolist()
color_to_index = {name: i for i, name in enumerate(color_names)}
n_colors = len(color_names)

In [ ]:
color_X = np.vstack(df.apply(lambda row: utils.artwork_to_vector(row, n_colors, color_to_index), axis=1))

In [ ]:
artist_encoder = LabelEncoder()
artist_X = artist_encoder.fit_transform(df["artist_full_name"]).reshape(-1, 1)

In [ ]:
school_encoder = LabelEncoder()
school_X = school_encoder.fit_transform(df["school"]).reshape(-1, 1)

In [ ]:
X = np.hstack([color_X, artist_X, school_X])

**2. Labels**

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df["decade"])

**3. Split dataset**

In [ ]:
X_train,  X_test,  y_train,  y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**4. Train Random Forest Classifier**

In [ ]:
clf = RandomForestClassifier(n_estimators=200, max_depth=20, random_state=42)
clf.fit(X_train, y_train)

In [ ]:
class_labels = label_encoder.inverse_transform(np.arange(len(label_encoder.classes_)))

**5. Predict**

In [ ]:
y_pred = clf.predict(X_test)

**6. Evaluate**

In [ ]:
labels_in_test = np.unique(y_test)

In [ ]:
class_names_in_test = [str(label_encoder.classes_[i]) for i in labels_in_test]

In [ ]:
# Generate report as dict
report_dict = classification_report(
    y_test, y_pred,
    labels=labels_in_test,
    target_names=class_names_in_test,
    zero_division=0,
    output_dict=True
)

In [ ]:
report = pd.DataFrame(report_dict).transpose()

In [ ]:
# Sort by support
summary_rows = report.loc[report.index.str.contains("avg|accuracy")]
class_rows = report.loc[~report.index.str.contains("avg|accuracy")].copy()
class_rows["support"] = class_rows["support"].astype(int)
class_rows_sorted = class_rows.sort_values(by="support", ascending=False)

In [ ]:
report = pd.concat([class_rows_sorted, summary_rows])

In [ ]:
report

In [ ]:
report.to_csv("weighted_decade_classifier_report.csv")

**7. Features importance**

In [ ]:
importances = clf.feature_importances_

In [ ]:
color_feature_names = color_names
artist_feature_name = ["artist"]
school_feature_name = ["school"]

feature_names = color_feature_names + artist_feature_name + school_feature_name

In [ ]:
importance_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values(by="importance", ascending=False)

In [ ]:
importance_df

In [ ]:
importance_df.to_csv("weighted_decade_feature_importance.csv", index=False)